# [Advent of Code 2020](https://adventofcode.com/2020)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [600]:
import operator
import re
from collections import defaultdict
from functools import partial, reduce
from itertools import cycle, islice, starmap


def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f'../data/advent2020/input{day}.txt'
    with open(filename) as fin:
        return mapt(parser, fin)

    
def mapt(fn, *args): 
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


def quantify(data, predictate=bool):
    "Count how many items in an iterable have predicated(item) True"
    return sum(map(predictate, data))


def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

cat = lambda s: ' '.join(s)

## [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

As usual, things start off easy enough, nested looping with a computationally simple upper bound. Finding numbers that add to 2020 and then returning their multiple.

In [441]:
def find_2020(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for n2 in nums[index:]:
            if n1 + n2 == 2020:
                return n1 * n2
    raise Exception('Unable to find solution')

In [442]:
data1 = Input(1, int)
find_2020(data1)

1006176

In [443]:
assert _ == 1006176, 'Day 1.1'

In [444]:
# part 2
def find_2020_3(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for jindex, n2 in enumerate(nums[index+1:]):
            for n3 in nums[jindex+1:]:
                if n1 + n2 + n3 == 2020:
                    return n1 * n2 * n3
    raise Exception('Unable to find solution')

find_2020_3(data1)

199132160

In [445]:
assert _ == 199132160, 'Day 1.2'

# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

In [446]:
def is_valid_password(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    occurance = quantify(password, lambda c: c == match)
    return lower <= occurance <= upper

In [447]:
def password_and_match(line):
    chunks = re.sub('[-:]', ' ', line).strip().split(' ')
    lower, upper, match, _, password = chunks
    return password, (match, int(lower), int(upper))

data2 = Input(2, password_and_match)
quantify(starmap(is_valid_password, data2))

600

In [448]:
assert _ == 600, 'Day 2.1'

In [449]:
# part 2
def is_valid_password2(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    is_match = lambda n: password[n - 1] == match
    matches = int(is_match(lower)) + int(is_match(upper))
    return matches == 1

quantify(starmap(is_valid_password2, data2))

245

In [450]:
assert _ == 245, 'Day 2.2'

# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

Here we can simply make use of some modulo arithmitic to loop back through the row when we attempt to index it out of bounds.

In [451]:
def count_trees(grid: [str], x_step: int, y_step: int):
    path = []
    for index, row in enumerate(grid[::y_step]):
        char_index = (x_step * index) % len(row)
        path.append(row[char_index])
    return quantify(path, lambda c: c == '#')
    

data3 = Input(3)
count_trees(data3, 3, 1)

151

In [452]:
assert _ == 151, 'Day 4.1'

In [453]:
# part 2
count_trees_3 = partial(count_trees, data3)
steps = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
reduce(
    operator.mul,
    starmap(count_trees_3, steps)
)

7540141059

In [454]:
assert _ == 7540141059, 'Day 3.2'

# [Day 4: Passport Processing](https://adventofcode.com/2020/day/4)

This one was a bit fiddly in terms of the regexes, I had an error thanks to not ensuring the match ended at the end of a line (i.e. missing a `$`.)

In [455]:
validators = {
    'byr': lambda x: 1920 <= int(re.match('\d{4}$', x)[0]) <= 2002,
    'iyr': lambda x: 2010 <= int(re.match('\d{4}$', x)[0]) <= 2020,
    'eyr': lambda x: 2020 <= int(re.match('\d{4}$', x)[0]) <= 2030,
    'hgt': lambda x: (
        150 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 193
        if re.match('\d+(cm|in)$', x)[1] == 'cm' else
        59 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 76
    ),
    'hcl': lambda x: bool(re.match('#[0-9a-f]{6}$', x)),
    'ecl': lambda x: bool(re.match('amb|blu|brn|gry|grn|hzl|oth$', x)),
    'pid': lambda x: bool(re.match('\d{9}$', x)),
    'cid': lambda x: True
}

def is_valid_passport(passport, validate_fields=False):
    required = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    if len(required - set([key for key, _ in passport])) > 0:
        return False

    if validate_fields:
        try:
            return all(
                validators[key](value)
                for key, value
                in passport
            )
        except TypeError:
            # regexes may not match, and will error if we attempt to index them
            return False
    return True


def parse_data(data):
    passports = [[]]
    for line in data:
        if not line.strip():
            # Move onto the next passport
            passports.append([])
            continue
        passports[-1].extend(
            re.findall(r'(\w+):([\w#\d]+)\b', line)
        )
        
    if not passports[-1]:
        passports.pop()
    
    return passports

In [456]:
input4 = parse_data(Input(4))
quantify(map(is_valid_passport, input4))

235

In [457]:
assert _ == 235, 'Day 4.1'

In [458]:
# part 2
is_valid_passport_strict = lambda passport: is_valid_passport(passport, True)
quantify(map(is_valid_passport_strict, input4))

194

In [459]:
assert _ == 194, 'Day 4.2'

# [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5)

Today's one invovled writing a reverse binary search, rather than looking for a value in a sorted list, we're instead given the steps to perform to find our seat.

For the second part we just loop through the list of sorted seat IDs until we find a gap, the gap being our seat.


In [460]:
def reverse_binary(steps: str, lower: int, upper: int) -> int:
    step = steps[0]
    pivot = lower + (upper - lower) // 2
    if step in ('F', 'L'):
        if len(steps) == 1:
            return lower
        return reverse_binary(steps[1:], lower, pivot)
    else:
        if len(steps) == 1:
            return upper
        return reverse_binary(steps[1:], pivot + 1, upper)

def docode_boarding_pass(boarding_pass: str) -> (int, int, int):
    row = reverse_binary(boarding_pass[:7], 0, 127)
    col = reverse_binary(boarding_pass[7:], 0, 7)
    seat_id = row * 8 + col
    return (row, col, seat_id)

In [461]:
input5 = Input(5)

decoded_passes = mapt(docode_boarding_pass, input5)
seat_ids = tuple(seat_id for _, _, seat_id in decoded_passes)
max(seat_ids)

926

In [462]:
assert _ == 926, 'Day 5.1'

In [463]:
# part 2
def find_missing_seat_id(seat_ids):
    sorted_ids = sorted(seat_ids)
    prev = sorted_ids[0]

    for seat_id in sorted_ids[1:]:
        if seat_id - 2 == prev:
            return seat_id - 1
        
        prev = seat_id

find_missing_seat_id(seat_ids)

657

In [464]:
assert _ == 657, 'Day 5.2'

# [Day 6: Custom Customs](https://adventofcode.com/2020/day/6)

Today's question was pretty straightforward, just involving some set-wise combinations. The answers I've given are quite terse (and in fact, slower than they're for-loop counterparts), but I enjoy them!

In [465]:
def group_answers(lines):
    groups = [[]]
    for line in lines:
        line = line.strip()
        if not line:
            groups.append([])
            continue
        groups[-1].append(line)
    
    if not groups[-1]:
        groups.pop()
    
    return groups

input6 = group_answers(Input(6))

In [500]:
sum(
    map(
        lambda d: len(reduce(operator.or_, map(set, d))),
        input6
    )
)

6662

In [467]:
assert _ == 6662, 'Day 6.1'

In [498]:
# part 2
sum(
    map(
        lambda d: len(reduce(operator.and_, map(set, d))),
        input6
    )
)

CPU times: user 2.06 ms, sys: 0 ns, total: 2.06 ms
Wall time: 2.07 ms


3382

In [469]:
assert _ == 3382, 'Day 6.2'

# [Day 7: Handy Haversacks](https://adventofcode.com/2020/day/7)

This is the first tree-like problem of the year! I've implemented an [adjacency list](https://en.wikipedia.org/wiki/Adjacency_list) to keep track of the relations, but this means having to invert the list for the first part.

In [601]:
class Node:
    def __init__(self, data):
        self.data = data
        self._children = []
        self.parent = None

    def add_child(self, child):
        self._children.append(child)
    
    def add_parent(self, parent):
        if self.parent:
            self.parent.remove_child(self)
        
        self.parent = parse_data

        if self.parent:
            self.parent.add_child(self)

    @property
    def root(self):
        if not self.parent:
            return self
        return self.parent.root
    

def parse_rules(rules):
    nodes = {}
    color_search = re.compile('(\d+) (\w+\s\w+) bags?,?\.?')
    for rule in rules:
        bag_colour = cat(rule.split(' ')[:2])
        parent_node = nodes.get(bag_colour, None)
        if not parent_node:
            parent_node = Node((0, bag_colour))
        for num, colour in color_search.findall(rule):
            data = (int(num), colour)
            node = Node(data)
            contents[bag_colour].append((int(num), colour))
    return contents


def count_allowed_bags(colour, rules):
    inverted_rules = defaultdict(list)

    for container_colour, allowed_bags in rules.items():
        for _, bag_colour in allowed_bags:
            inverted_rules[bag_colour].append(container_colour)

    allowed = set()
    to_follow = inverted_rules[colour][:]

    while to_follow:
        allowed |= set(to_follow)
        old = to_follow[:]
        to_follow = []
        for c in old:
            to_follow.extend(inverted_rules[c])

    return len(allowed)

In [602]:
input7 = parse_rules(Input(7))
count_allowed_bags('shiny gold', input7)

370

In [584]:
assert _ == 370, 'Day 7.1'

In [595]:
# part 2
def count_required_bags(initial_colour, rules):
    to_follow = [(1, initial_colour)]
    required = 0

    while to_follow:
        num, colour = to_follow.pop(0)
        
        for next_rule in rules[colour]:
            # rather than duplicate the rule num times, just duplicate
            # the number of bags
            next_num, next_bag_colour = next_rule
            to_follow.append((next_num * num, next_bag_colour))
        required += num

    # the amount will be off by 1 as we count the initial bag
    return required - 1

In [604]:
count_required_bags('shiny gold', input7)

29547

In [605]:
assert _ == 29547, 'Day 7.2'